In [1]:
import assembly_game
import gymnasium as gym

game = gym.make("Min2Game")

game.reset()

([1, 2, 0, 0, 2, 1, 0, 0], {})

Let's first move the value from %rdi to the return value (%rax), as you can see we immediately get reward of 10 because in one training example result is in the correct spot \
we are getting however the penalty of 1 for every timestep.

In [5]:
from assembly_game.processor import Processor

list(enumerate(Processor.actions))

[(0, (<Instruction.MOV: 0>, <Operand.RDI: 0>, <Operand.RDI: 0>)),
 (1, (<Instruction.MOV: 0>, <Operand.RDI: 0>, <Operand.RSI: 1>)),
 (2, (<Instruction.MOV: 0>, <Operand.RDI: 0>, <Operand.RAX: 2>)),
 (3, (<Instruction.MOV: 0>, <Operand.RSI: 1>, <Operand.RDI: 0>)),
 (4, (<Instruction.MOV: 0>, <Operand.RSI: 1>, <Operand.RSI: 1>)),
 (5, (<Instruction.MOV: 0>, <Operand.RSI: 1>, <Operand.RAX: 2>)),
 (6, (<Instruction.MOV: 0>, <Operand.RAX: 2>, <Operand.RDI: 0>)),
 (7, (<Instruction.MOV: 0>, <Operand.RAX: 2>, <Operand.RSI: 1>)),
 (8, (<Instruction.MOV: 0>, <Operand.RAX: 2>, <Operand.RAX: 2>)),
 (9, (<Instruction.CMP: 1>, <Operand.RDI: 0>, <Operand.RDI: 0>)),
 (10, (<Instruction.CMP: 1>, <Operand.RDI: 0>, <Operand.RSI: 1>)),
 (11, (<Instruction.CMP: 1>, <Operand.RDI: 0>, <Operand.RAX: 2>)),
 (12, (<Instruction.CMP: 1>, <Operand.RSI: 1>, <Operand.RDI: 0>)),
 (13, (<Instruction.CMP: 1>, <Operand.RSI: 1>, <Operand.RSI: 1>)),
 (14, (<Instruction.CMP: 1>, <Operand.RSI: 1>, <Operand.RAX: 2>)),
 (15,

In [ ]:
game.step(2) # MOV %rdi, %rax

([1, 2, 1, 0, 2, 1, 2, 0], 9, False, False, {})

Let's now compare the value with the value in %rsi, if $rax happens to be greater then it must be the case that the value %rsi is minimum

In [7]:
game.step(14) # CMP $rsi, %rax

([1, 2, 1, -1, 2, 1, 2, 1], 8, False, False, {})

After conditional move all testing examples are "solved", as we are getting reward of 20 and penalty of 3 instructions used

In [8]:
game.step(23) # CMOVG %rsi, %rax

([1, 2, 1, -1, 2, 1, 1, 1], 17, False, False, {})

Lastly we need to run the RET instructions to observe value of terminated=True

In [9]:
game.step(27) # RET

([1, 2, 1, -1, 2, 1, 1, 1], 16, True, False, {})

# Now let's see how to add timelimit to the environment

In [10]:
game = gym.make("Min2Game", max_episode_steps=4)

In [ ]:
game.reset()
DUMMY = (0)
game.step(DUMMY)

([(1, 2, 1, 0), (2, 1, 2, 0)], 9, False, False, {})

After 4 steps the truncated is being set to True, indicating that the episode has ended due to the time limit.

In [18]:
game.step(DUMMY)
game.step(DUMMY)
game.step(DUMMY)

([(1, 2, 1, 0), (2, 1, 2, 0)], 4, False, True, {})